<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [1]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 19.3MB/s]


### Import the required libraries

In [2]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [3]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [4]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [6]:
target_cols = data.select_dtypes(include='object').columns
target_cols

Index(['Brand', 'Model', 'FastCharge_KmH', 'RapidCharge', 'PowerTrain',
       'PlugType', 'BodyStyle', 'Segment'],
      dtype='object')

In [7]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
df_copy = data.copy()
for col in target_cols:
  df_copy[col] = enc.fit_transform(df_copy[col])

In [8]:
df_copy.sample(10)

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
88,27,27,6.2,180,400,193,30,1,0,2,6,2,5,47500
62,20,7,7.6,160,440,198,29,1,1,2,1,2,5,50000
95,31,31,7.3,160,340,171,24,1,2,2,1,2,5,35575
53,27,25,8.8,160,420,183,32,1,2,2,6,2,5,40000
47,24,65,4.0,250,365,195,39,1,0,2,7,5,4,102945
58,5,82,9.7,150,250,180,16,1,1,2,6,2,5,40000
56,6,1,8.7,150,250,180,16,1,1,2,6,1,5,37422
33,30,16,3.0,210,750,267,38,1,0,2,4,6,6,75000
63,11,28,4.8,200,365,232,13,1,0,2,6,4,5,75351
20,21,13,8.1,150,275,164,19,1,1,2,1,1,5,29146


## Split the dataset for training and testing in ratio 80:20

In [13]:
from sklearn.model_selection import train_test_split

x_data = df_copy.drop(columns=['PriceEuro'], axis=1)
y_data = df_copy['PriceEuro']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

DecisionTreeRegressor()

## Predict the outcomes for X test

In [31]:
y_pred_dtr = model.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [32]:
from sklearn.metrics import r2_score

In [33]:
r2 = r2_score(y_pred_dtr, y_test)
print(r2)

0.7781568738324933


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [34]:
from xgboost import XGBRegressor

In [35]:
model = XGBRegressor()
model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [36]:
y_pred = model.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [37]:
r2 = r2_score(y_pred, y_test)
print(r2)

0.8824533224105835


## Compare the performances of both model for at least three regression metircs

In [38]:
from sklearn.metrics import *
metrics = {
    "MAE": mean_absolute_error,
    "MSE": mean_squared_error,
    "R2": r2_score
}

print("Model Performance Comparison:")
for metric_name, metric_func in metrics.items():
    score_1 = metric_func(y_test, y_pred_dtr)
    score_2 = metric_func(y_test, y_pred)
    print(f"{metric_name}: Model 1 = {score_1:.2f}, Model 2 = {score_2:.2f}")

Model Performance Comparison:
MAE: Model 1 = 7058.33, Model 2 = 5892.85
MSE: Model 1 = 149398042.33, Model 2 = 85413432.58
R2: Model 1 = 0.84, Model 2 = 0.91
